In [1]:
import pandas as pd
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import torch

from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import MAE, SMAPE, MultivariateNormalDistributionLoss

/home/corcasta/miniconda3/envs/farama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("output.csv")

In [3]:
data

,Age,Male,Female,Quantity,Total_Amount,Product_ID,Product_Category,Price_per_Unit,Time_Unitless,Year,Month,Week,Window_Mean_4,Window_Mean_5,Window_Mean_6,Window_Mean_7
0,19.0,0,1,1,25,0,Beauty,25,0,2023,1,1,81.25,80.0,70.83,78.57
1,19.0,1,0,2,50,0,Beauty,25,1,2023,1,2,81.25,80.0,70.83,78.57
2,34.0,1,2,10,250,0,Beauty,25,2,2023,1,3,81.25,80.0,70.83,78.57
3,0.0,0,0,0,0,0,Beauty,25,3,2023,1,4,81.25,80.0,70.83,78.57
4,23.0,0,1,3,75,0,Beauty,25,4,2023,1,5,93.75,80.0,70.83,78.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,21.0,2,0,6,3000,14,Electronics,500,45,2023,11,47,1625.00,1700.0,1750.00,2142.86
772,0.0,0,0,0,0,14,Electronics,500,46,2023,11,48,1625.00,1300.0,1416.67,1500.00
773,55.0,0,1,3,1500,14,Electronics,500,47,2023,12,49,1125.00,1600.0,1333.33,1428.57
774,50.0,4,1,12,6000,14,Electronics,500,48,2023,12,50,2625.00,2100.0,2333.33,2000.00


In [4]:
input_features = list(data.columns.drop(["Product_ID", "Total_Amount", "Product_Category", "Time_Unitless"]))
feature_scalers = [None]*len(input_features)
scalers_dict = dict(zip(input_features, feature_scalers))

In [5]:
test_df = data[data["Product_ID"] == 0]

In [6]:
test_df.Total_Amount

0      25
1      50
2     250
3       0
4      75
5      25
6     125
7       0
8     125
9     200
10     25
11      0
12      0
13      0
14      0
15     25
16    375
17    125
18     75
19    100
20     50
21     75
22     25
23      0
24      0
25      0
26    100
27      0
28    100
29    100
30      0
31     50
32      0
33      0
34    250
35     25
36    200
37     50
38      0
39    200
40      0
41    100
42    200
43    250
44      0
45     25
46    100
47    150
48     25
49    175
50     50
51     25
Name: Total_Amount, dtype: int64

In [7]:
max_encoder_length = 12
max_prediction_length = 8

shortest_series_len = data.groupby(["Product_ID"])["Time_Unitless"].max().min()

# This will indicate what will be the training dataset
# And what will be the validation dataset
training_cutoff = shortest_series_len - max_prediction_length


train_dataset = TimeSeriesDataSet(
    data[lambda x: x.Time_Unitless <= training_cutoff],
    time_idx="Time_Unitless",
    target="Total_Amount",
    target_normalizer=None,
    categorical_encoders={"Product_Category": NaNLabelEncoder().fit(data.Product_Category)},
    group_ids=["Product_ID"],
    static_categoricals=[
        "Product_Category"
    ],  # as we plan to forecast correlations, it is important to use series characteristics (e.g. a series identifier)
    time_varying_unknown_reals=["Total_Amount", *input_features],
    min_encoder_length=12,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    scalers=scalers_dict,
)

valid_dataset = TimeSeriesDataSet.from_dataset(train_dataset, data, min_prediction_idx=training_cutoff + 1)

In [8]:
train_dataloader = train_dataset.to_dataloader(batch_size=4, train=True)

valid_dataloader = valid_dataset.to_dataloader(
    train=False, batch_size=4, num_workers=0)#, batch_sampler="synchronized"


In [9]:
train_dataset.get_parameters()

{'time_idx': 'Time_Unitless',
 'target': 'Total_Amount',
 'group_ids': ['Product_ID'],
 'weight': None,
 'max_encoder_length': 12,
 'min_encoder_length': 12,
 'min_prediction_idx': 0,
 'min_prediction_length': 8,
 'max_prediction_length': 8,
 'static_categoricals': ['Product_Category'],
 'static_reals': None,
 'time_varying_known_categoricals': None,
 'time_varying_known_reals': None,
 'time_varying_unknown_categoricals': None,
 'time_varying_unknown_reals': ['Total_Amount',
  'Age',
  'Male',
  'Female',
  'Quantity',
  'Price_per_Unit',
  'Year',
  'Month',
  'Week',
  'Window_Mean_4',
  'Window_Mean_5',
  'Window_Mean_6',
  'Window_Mean_7'],
 'variable_groups': None,
 'constant_fill_strategy': None,
 'allow_missing_timesteps': False,
 'lags': None,
 'add_relative_time_idx': False,
 'add_target_scales': False,
 'add_encoder_length': False,
 'target_normalizer': TorchNormalizer(method='identity', center=True, transformation=None, method_kwargs=None),
 'categorical_encoders': {'Product

In [10]:
from pytorch_forecasting import Baseline, NHiTS, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import MAE, SMAPE, MQF2DistributionLoss, QuantileLoss

In [11]:
pl.seed_everything(42)
trainer = pl.Trainer(accelerator="cpu", gradient_clip_val=0.1)
net = NHiTS.from_dataset(
    train_dataset,
    learning_rate=3e-2,
    weight_decay=1e-2,
    loss=MQF2DistributionLoss(prediction_length=max_prediction_length),
    backcast_loss_ratio=0.0,
    hidden_size=64,
    optimizer="AdamW",
)

Seed set to 42
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/corcasta/miniconda3/envs/farama/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/home/corcasta/miniconda3/envs/farama/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/corcasta/miniconda3/envs/farama/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.sav

In [12]:
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    net, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader, min_lr=1e-5, max_lr=1e-1
)
print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()
net.hparams.learning_rate = res.suggestion()

/home/corcasta/miniconda3/envs/farama/lib/python3.10/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [38]:
x, y = next(iter(train_dataloader))